# FSK Modulation and Demodulation

FSK modulation varies the frequency of a signal to modulate information, and can be demodulated using techniques such as a Goertzel filter. FSK is insensitive to non-frequency-dependent amplitude variations.

In [1]:
import numpy as np
from scipy import signal

## Modulation

We use coherent FSK modulation to prevent discontinuities when the frequency changes. To accomplish this, we view frequency as the derivative of phase, first constructing the list of frequencies, and then integrating it over time to get the instantaneous phase.

In [2]:
# Plot

We also do amplitude shaping at the ends by padding the ends with a 0-frequency dummy symbol and performing Gaussian smoothing on the amplitude to prevent discontinuity at the ends of the modulated signal.

## Demodulation

To perform demodulation, we construct a Goertzel filter, tuned to the desired frequency. The form of the Goertzel filter (taken from https://www.dsprelated.com/showarticle/796.php) is

\begin{align*}
H_G(z) &= \frac{1}{1-e^{2\pi jf}z^{-1}} \cdot \frac{1-e^{-2\pi jf}z^{-1}}{1-e^{-2\pi jf}z^{-1}} \\
&= \frac{1-e^{-2\pi jf}z^{-1}}{1-2\cos(2\pi f)z^{-1}+z^{-2}}
\end{align*}

We use this form (which has complex-conjugate poles and a zero to cancel out the bottom pole) to improve numerical stability, as explained in the linked article above.

In [3]:
def goertzel_iir(freq,fs):
    if freq>=0.5*fs:
        raise ValueError("Desired peak frequency is too high")
    norm_freq=2*np.pi*freq/fs
    numerator=[1,-np.exp(-1j*norm_freq)]
    denominator=[1,-2*np.cos(norm_freq),1]
    return (numerator,denominator)

In order to perform the demodulation, we divide the modulated signal into symbol blocks, filter each block through Goertzel filters tuned to each frequency, and compare the absolute values of the last output of each filter.

In [4]:
# Compute and plot